Follow this [Abu Bakr Soliman](https://colab.research.google.com/drive/1tf6pjAwz2ZJdv1IcP6YcIrLFa4iW6svT?) for more data and more explanations

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [1]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install -qU faker==35.2.0
!pip install -qU vllm==0.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 M

In [58]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 356, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 356 (delta 76), reused 267 (delta 66), pack-reused 0 (from 0)
Receiving objects: 100% (356/356), 9.72 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wand'))
hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmadhammad to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Fine-tuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Fine-tuning`


In [4]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


data_dir = "/gdrive/MyDrive/llms/llm-finetuning"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
device = "cuda"

torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [5]:
story = """
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
المتعة والراحة. ومع ذلك، قد يصبح
البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
 قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

 كيف تؤثر العائلة على علاقتنا بالمال؟
يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
 "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
(Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

تتضمن هذه الأداة:

رسم شجرة عائلية.
تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
على سبيل المثال، إذا نشأ شخص في عائلة
اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
 أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
"""

In [8]:
story = """
خبرني - بدأت أفواج الحجاج الأردنيين وحجاج مسلمي 48 بالتوجه إلى الأراضي المقدسة عبر معبر المدورة الحدودي، استعداداً لأداء مناسك الحج لهذا العام.

ويبلغ إجمالي عدد الحجاج الأردنيين هذا الموسم 8 آلاف حاج، إضافة إلى 4500 حاج من مسلمي الـ 48.

ومن المتوقع أن يغادر قرابة 600 حاج وحاجة يوم غد، السبت، متوجهين إلى الأراضي المقدسة للانضمام إلى أفواج الحجيج.

وفي سياق متصل، سيقوم القائم بأعمال السفارة بالإنابة، الأستاذ محمد مؤنس، بتوديع الحجاج يوم غد السبت، بحضور وزير الأوقاف والشؤون والمقدسات الإسلامية محمد الخلايلة، في ساحة مدينة حجاج الجيزة الساعة الواحدة مساءً.
"""

In [9]:
# {
#     "story_title": "",
#     "story_keywords": ["kw1", "kw2"],
#     "story_summary": [""],
#     "story_category": "",
#     "story_entities": []
# }
# convert the data into a schema

StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]
class Entity(BaseModel):
  entity_value: str = Field(..., description="The actual name or value of the entity")
  entity_type: EntityType = Field(..., description="The type of the entity")

class NewsDetails(BaseModel):
  story_title: str = Field(..., min_length=5, max_length=300, description="A fully informative and SEO optimized title of the story")
  story_keywords: List[str] = Field(..., min_items=1,
                                    description="Relevant keywords associated with the story")

  story_summary: List[str] = Field(..., min_items=1, max_items=5,
                                   description="A concise summary of the story")

  story_category: StoryCategory = Field(..., description="category of the news story")

  story_entities: List[Entity] = Field(..., min_items=1, max_items=10,
                                       description="Entities mentioned in the story")

In [10]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                NewsDetails.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Story Details:",
            "```json"
        ])
    }
]


## Translation

In [11]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., min_length=5, max_length=300,
                                  description="Suggested translated title of the news story.")
    translated_content: str = Field(..., min_length=5,
                                    description="Translated content of the news story.")

targeted_lang = "English"

translation_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content":  "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
            "",

            "## Targeted Language:",
            targeted_lang,
            "",

            "## Translated Story:",
            "```json"

        ])
    }
]

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [15]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [23]:
print(response)

{
  "story_title": "How Family Shapes Our Financial Relationships",
  "story_keywords": [
    "family influence on money",
    "financial relationships",
    "inheritance patterns",
    "moneymaking habits",
    "personal finance"
  ],
  "story_summary": [
    "Famous magazine Forbes stated that family plays a crucial role in shaping individuals' financial relationships.",
    "This relationship is influenced by inherited behavioral patterns across generations."
  ],
  "story_category": "economy",
  "story_entities": [
    {
      "entity_value": "Forbes",
      "entity_type": "organization"
    },
    {
      "entity_value": "Shane Einot",
      "entity_type": "person-male"
    },
    {
      "entity_value": "Financial Therapy Association",
      "entity_type": "organization"
    }
  ]
}


In [24]:
text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

{
  "translated_title": "Forbes Magazine Reveals Family Plays a Central Role in Forming Individuals' Financial Relationships",
  "translated_content": "According to Forbes magazine, family plays a crucial role in shaping individuals' financial relationships, as these relationships are influenced by inherited behavioral patterns across generations."
}


In [25]:
from openai import OpenAI
from google.colab import userdata

openai_client = OpenAI(
    api_key=userdata.get('openai-colab'),
    # base_url="http://localhost:8000"
)

openai_model_id = "gpt-4o-mini"

In [27]:
chat_completion = openai_client.chat.completions.create(
    messages=details_extraction_messages,
    model=openai_model_id,
    temperature=0.2,
)

print(chat_completion.choices[0].message.content)

{
  "story_title": "تأثير العائلة على العلاقة بالمال",
  "story_keywords": [
    "العائلة",
    "الشخصية المالية",
    "المال",
    "السلوك المالي",
    "التخطيط المالي"
  ],
  "story_summary": [
    "تستعرض الدراسة تأثير العائلة على العلاقة بالمال.",
    "تحدد الدراسة ثلاثة أبعاد رئيسية للعلاقة بالمال.",
    "توضح كيف تؤثر التجارب الأسرية على الشخصية المالية.",
    "تقدم رابطة العلاج المالي أداة لتحليل الأنماط المالية.",
    "تسجل النتائج تأثيرات السلوك المالي على الأفراد."
  ],
  "story_category": "economy",
  "story_entities": [
    {
      "entity_value": "مجلة فوربس",
      "entity_type": "organization"
    },
    {
      "entity_value": "شاين إنيت",
      "entity_type": "person-male"
    },
    {
      "entity_value": "رابطة العلاج المالي",
      "entity_type": "organization"
    },
    {
      "entity_value": "مخطط الجينوم المالي",
      "entity_type": "product"
    }
  ]
}


In [28]:
chat_completion = openai_client.chat.completions.create(
    messages=translation_messages,
    model=openai_model_id,
    temperature=0.2,
)

print(chat_completion.choices[0].message.content)

{
  "translated_title": "The Impact of Family on Financial Relationships",
  "translated_content": "Forbes magazine reported that family plays a pivotal role in shaping individuals' relationships with money, as this relationship is influenced by financial behavior patterns passed down through generations.\n\nThe report, based on research by university professor Shane Enright on financial well-being, explains that each person has a \"financial personality\" determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\n\nThe three dimensions of the relationship with money\nAccording to the study, there are three main dimensions that shape our relationship with money:\n\nAcquisition (A): Individuals belonging to this dimension tend to view money as a commodity to be amassed, seeing wealth accumulation as an end in itself. The downside of this pattern is the potential to develop an obsession with wealth or, conversely, a complete re

In [29]:
json.loads(chat_completion.choices[0].message.content)

{'translated_title': 'The Impact of Family on Financial Relationships',
 'translated_content': 'Forbes magazine reported that family plays a pivotal role in shaping individuals\' relationships with money, as this relationship is influenced by financial behavior patterns passed down through generations.\n\nThe report, based on research by university professor Shane Enright on financial well-being, explains that each person has a "financial personality" determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\n\nThe three dimensions of the relationship with money\nAccording to the study, there are three main dimensions that shape our relationship with money:\n\nAcquisition (A): Individuals belonging to this dimension tend to view money as a commodity to be amassed, seeing wealth accumulation as an end in itself. The downside of this pattern is the potential to develop an obsession with wealth or, conversely, a complete rejecti

## Knowledge distillation

In [50]:
raw_data_path = join(data_dir, "datasets", "news-sample.jsonl")

raw_data = []
for line in open(raw_data_path):
    if line.strip() == "":
        continue

    raw_data.append(
        json.loads(line.strip())
    )

random.Random(101).shuffle(raw_data)

print(f"Raw data: {len(raw_data)}")

Raw data: 2400


In [51]:
raw_data[0]

{'id': 975,
 'title': 'ما تقدمه فلسطين للعالم.. معرض لآمال وآلام شعبها في باريس',
 'description': 'يواصل المعهد العربي في باريس استقبال زواره في معرض “ما تقدمه فلسطين للعالم” لإطلاعهم على الإرث الثقافي والفني للفلسطينيين، من خلال أعمال فنية لآمالهم، وصور لواقعهم الأليم تحت الاحتلال.',
 'content': 'يواصل المعهد العربي في باريس استقبال زواره في معرض ما تقدمه فلسطين للعالم لإطلاعهم على الإرث الثقافي والفني للفلسطينيين؛ من خلال أعمال فنية لآمالهم وصور لواقعهم الأليم تحت الاحتلال. \n ويرى رئيس المعهد جاك لانغ -الذي أُعيد انتخابه قبل أيام للدورة الرابعة- ما يحدث في غزة حاليا جراء العدوان الإسرائيلي أنه كارثة. \n والمعهد هو مركز ثقافي وواجهة دبلوماسية يديرها لانغ منذ 2013 ويقع على ضفة نهر السين في باريس. \n وأشار لانغ، الذي شغل سابقا منصب وزير الثقافة بفرنسا، إلى أن المعرض هو إهداء للشعب الفلسطيني، ومُدّد ليستقبل مزيدا من الزوار حتى 31 ديسمبركانون الأول الجاري. \n ويضم المعرض، الذي افتُتح أواخر مايوأيار الماضي، حسب لانغ العديد من المعارض الفرعية عن فلسطين وعن غزة بالتحديد، من بينها معرض الصور

In [54]:
cloud_model_id = "gpt-4o-mini"
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0

save_to = join(data_dir, "datasets", "sft.jsonl")

ix = 0
for story in tqdm(raw_data):

    sample_details_extraction_messages = [
        {
            "role": "system",
            "content": "\n".join([
                "You are an NLP data paraser.",
                "You will be provided by an Arabic text associated with a Pydantic scheme.",
                "Generate the ouptut in the same story language.",
                "You have to extract JSON details from text according the Pydantic details.",
                "Extract details as mentioned in text.",
                "Do not generate any introduction or conclusion."
            ])
        },
        {
            "role": "user",
            "content": "\n".join([
                "## Story:",
                story['content'].strip(),
                "",

                "## Pydantic Details:",
                json.dumps(
                    NewsDetails.model_json_schema(), ensure_ascii=False
                ),
                "",

                "## Story Details:",
                "```json"
            ])
        }
    ]

    response = openai_client.chat.completions.create(
                            messages=sample_details_extraction_messages,
                            model=cloud_model_id,
                            temperature=0.2,
                        )

    if response.choices[0].finish_reason != "stop":
        prompt_tokens += response.usage.prompt_tokens
        continue

    llm_response = response.choices[0].message.content
    llm_resp_dict = parse_json(llm_response)

    if not llm_resp_dict:
        continue

    with open(save_to, "a", encoding="utf8") as dest:
        dest.write(json.dumps({
            "id": ix,
            "story": story['content'].strip(),
            "task": "Extrat the story details into a JSON.",
            "output_scheme": json.dumps( NewsDetails.model_json_schema(), ensure_ascii=False ),
            "response": llm_resp_dict,
        }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1
    prompt_tokens += response.usage.prompt_tokens
    completion_tokens += response.usage.completion_tokens

    if(ix % 3) == 0:
        cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
        cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
        total_cost = cost_input + cost_output

        print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

  0%|          | 0/2400 [00:00<?, ?it/s]

Iteration 3: Total Cost = $0.0014 
Iteration 6: Total Cost = $0.0027 
Iteration 9: Total Cost = $0.0041 
Iteration 12: Total Cost = $0.0053 
Iteration 15: Total Cost = $0.0068 


KeyboardInterrupt: 

## Format Finetuning Datasets

In [55]:

sft_data_path = join(data_dir, "datasets", "sft.jsonl")
llm_finetunning_data = []

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Story:",
            rec["story"],

            "# Task:",
            rec["task"],

            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)

In [57]:
len(llm_finetunning_data)

2782

In [60]:
train_sample_sz = 2700

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [61]:
join(data_dir, "datasets", "llamafactory-finetune-data", "val.json")

'/gdrive/MyDrive/llms/llm-finetuning/datasets/llamafactory-finetune-data/val.json'

## Finetuning

In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
  "news_finetune_train": {
      "file_name": "/gdrive/MyDrive/youtube-resources/llm-finetuning/datasets/llamafactory-finetune-data/train.json",
      "columns": {
          "prompt": "instruction",
          "query": "input",
          "response": "output",
          "system": "system",
          "history": "history"
      }
  },
  "news_finetune_val": {
      "file_name": "/gdrive/MyDrive/youtube-resources/llm-finetuning/datasets/llamafactory-finetune-data/val.json",
      "columns": {
          "prompt": "instruction",
          "query": "input",
          "response": "output",
          "system": "system",
          "history": "history"
      }
  }
# ```

# https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

In [1]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
# cutoff_len: 3500
max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/youtube-resources/llm-finetuning/models/checkpoint-1500
output_dir: /gdrive/MyDrive/llms/llm-finetuning/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

# push_to_hub: true
# export_hub_model_id: "ahmadhammad/news-analyzer"
# hub_private_repo: true
# hub_strategy: checkpoint

Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [2]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

2025-05-24 12:20:02.060158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748089202.108828   36286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748089202.119258   36286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-24 12:20:02.154501: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[INFO|2025-05-24 12:20:10] llamafactory.hparams.parser:401 >> Process rank: 0, world size: 1, device: cuda:0, distrib

## New fine tuned model

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
finetuned_model_id = "/gdrive/MyDrive/llms/llm-finetuning/models"
model.load_adapter(finetuned_model_id)

In [14]:
def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_resp(translation_messages)
parse_json(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'translated_title': "Jordanian Hajj Pilgrims and Muslims from the 48th Province Depart for Holy Land via Mada'in Border Crossing",
 'translated_content': "The Jordanian pilgrims and Muslim pilgrims from the 48th province began their journey to the Holy Land through the Mada'in border crossing, preparing for this year's Hajj rituals.\n\nThis season, the total number of Jordanian pilgrims is estimated at 8,000, in addition to 4500 pilgrims from the 48th province.\n\nIt is expected that approximately 600 pilgrims and pilgrims will depart tomorrow, Sunday, heading towards the Holy Land to join the pilgrim groups."}

## Cost estimation

In [16]:
from tqdm.auto import tqdm
from faker import Faker
import random
from datetime import datetime

start_time = datetime.now()
fake = Faker('ar')

input_tokens = 0
output_tokens = 0

for i in tqdm(range(5)):
    prompt = fake.text(max_nb_chars=random.randint(150, 200))

    messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]

    response = generate_resp(messages)

    input_tokens += len(tokenizer.apply_chat_template(messages))
    output_tokens += len(tokenizer.encode(response))

total_time = (datetime.now() - start_time).total_seconds()

print(f"Total Time: {total_time} seconds")
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {input_tokens + output_tokens}")

  0%|          | 0/5 [00:00<?, ?it/s]

Total Time: 53.615707 seconds
Input Tokens: 394
Output Tokens: 826
Total Tokens: 1220


## VLLM

In [ ]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_model_id = "/gdrive/MyDrive/llms/llm-finetuning/models"


!nohup vllm serve "{base_model_id}" --dtype=half --gpu-memory-utilization 0.8 --max_lora_rank 64 --enable-lora --lora-modules news-lora="{adapter_model_id}" &

In [ ]:
!tail -n 30 nohup.out

## Inference

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

prompt = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
vllm_model_id = "news-lora"

llm_response = requests.post("http://localhost:8000/v1/completions", json={
    "model": vllm_model_id,
    "prompt": prompt,
    "max_tokens": 1000,
    "temperature": 0.3
})

llm_response.json()

## Load testing

In [ ]:
%%writefile locust.py

import random
import json
from locust import HttpUser, task, between, constant
from transformers import AutoTokenizer
from faker import Faker

fake = Faker('ar')

class CompletionLoadTest(HttpUser):
    wait_time = between(1, 3)

    @task
    def post_completion(self):
        model_id = "news-lora"
        prompt = fake.text(max_nb_chars=random.randint(150, 200))

        message = {
            "model": model_id,
            "prompt": prompt,
            "max_tokens": 512,
            "temperature": 0.3
        }

        llm_response = self.client.post("/v1/completions", json=message)

        if llm_response.status_code == 200:
            with open("./vllm_tokens.txt", "a") as dest:
                dest.write(json.dumps({
                    "prompt": prompt,
                    "response": llm_response.json()["choices"][0]["text"],
                }, ensure_ascii=False) + "\n")


In [ ]:
!locust --headless -f locust.py --host=http://localhost:8000 -u 20 -r 1 -t "60s" --html=locust_results.html

In [ ]:
vllm_tokens = [
    json.loads(line.strip())
    for line in open("./vllm_tokens.txt") if line.strip() != ""
]

In [ ]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

total_input_tokens = sum([ len(tokenizer.encode(rec['prompt'])) for rec in vllm_tokens ])
total_output_tokens = sum([ len(tokenizer.encode(rec['response'])) for rec in vllm_tokens ])

print(f"Total Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")

In [ ]:
37840 / 60